# 1D Linear FE Classes

This notebook contains classes for the solution of steady, 2nd order, linear ODE's in one dimension. The code uses Lagrange interpolation polynomials. It includes an example solution of the steady-state homogeneous heat equation with conduction and advection terms.

The code is based on the discussion in a set of presentation [slides](http://www.math.pitt.edu/~sussmanm/3040Summer14/FEM1D.pdf) by M.M. Sussman, University of Pittsburgh, accessed 23/02/2022. The slides are incomplete and were used mainly as a guide to build a class architecture for the problem. The lecture notes in turn are based on code from the [FEniCS project](https://fenicsproject.org). Notebook author: Mattis Voss

In [ ]:
# Install helper libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Image

"""Code based on and adapted from presentation slides by M.M. Sussman, 
University of Pittsburgh.
URL: http://www.math.pitt.edu/~sussmanm/3040Summer14/FEM1D.pdf
Accessed 23/02/2022. Notebook author: Mattis Voss"""

## Basis Functions

In [ ]:
class BasisFunctions1D:
    ''' Define set of Lagrange basis functions on (local) interval [0,1].
    The local spatial coordinate over the element is xi 
    (as in the Greek letter).
    Order can be 1 (for pair of linear functions) or 2 
    (for triple of quadratic functions)
    
    PUBLIC METHODS    
        eval(n, xi): evaluate basis function phi[n](xi)
        dx(n, xi): evaluate first derivative wrt xi 
        of basis function dphi[n](xi)
        size(): return number of nodes of element
        plot(): plot all basis functions and their derivatives wrt xi'''
    
    def __init__(self, order):
        '''
        PARAMETERS
            order: order of basis functions to use (linear or quadratic)
        '''
        self.__order = order
        if self.__order == 1:
            # Vector of basis functions
            self.__phi = np.array([lambda xi: 1 - xi, 
                                   lambda xi: xi])
            # Vector of derivates w.r.t. xi of basis functions
            self.__dphi_dxi = np.array([lambda xi: -1*np.ones_like(xi), 
                                       lambda xi: 1*np.ones_like(xi)])
        elif self.__order == 2:
            # Vector of basis functions
            self.__phi = np.array([lambda xi: 2.0 * (xi-0.5) * (xi-1.0),
                                    lambda xi: 4.0 * xi * (1.0-xi),
                                    lambda xi: 2.0 * xi * (xi-0.5)])
            # Vector of derivates w.r.t. xi of basis functions
            self.__dphi_dxi = np.array([lambda xi: 2.0 * (xi-0.5) +
                                        2.0*(xi - 1.0),
                                        lambda xi: -4.0 * xi + 4.0*(1.0 - xi),
                                        lambda xi: 2.0 * xi + 2.0*(xi - 0.5)])
        else:
            raise Exception("Polynomial order must be 1 or 2")
        
        # Number of nodes (always 2 for 1D linear basis functions, 
        # 3 for 1D quadratic basis functions)
        self.__N = order + 1 
        
    def eval(self,n,xi):
        '''
        Evaluate the function phi
        PARAMETERS
            n := index of the basis function
            xi := vector of values of xi at which to evaluate
        RETURNS
            Vector of values of basis function phi
        '''
        return self.__phi[n](xi)
    
    def dx(self,n,xi):
        '''
        RETURNS: derivate of the function phi w.r.t. x
        '''
        return self.__dphi_dxi[n](xi)
    
    def size(self):
        '''
        RETURNS: number of nodes (1 greater than order of 
        interpolation polynomial)
        '''
        return self.__N
    
    def order(self):
        ''' RETURNS: order of interpolation polynomial '''
        return self.__order
    

    def plot(self,xi): 
        '''
        Plot all basis functions of the current instance, and their 
        first derivatives
        '''
        fig, (ax1, ax2) = plt.subplots(2,1)
        fig.suptitle(f'Local Lagrange polynomial basis functions \
                     of order {self.__order}')       
        # Set axis labels
        ax1.set_ylabel(r'$\phi(\xi)$')
        ax1.set_xlabel(r'$\xi$')
        ax2.set_xlabel(r'$\xi$')
        ax2.set_ylabel(r'$\phi^\prime(\xi)$')

        for i in range(self.__N):
            ax1.plot(xi, self.__phi[i](xi))
            ax2.plot(xi, self.__dphi_dxi[i](xi))
        plt.show()

## Mesh

In [ ]:
class Mesh1D:
    '''Define a 1D FE mesh. Note only linear elements available for now.
    
    PUBLIC METHODS
        all_nodes(): return array of all node numbers
        cell_nodes(e): return node numbers of cell e
        all_coords(): return coordinates (global x-values) of all nodes
        cell_coords(e): return coordinates of nodes of cell e
        node_coord(n): return coordinate (global x-value) of node n
        node_coords(): return array of all node coordinates
        size(): return number of elements in mesh
        cells(): return ordered array of cell numbers
        nnodes(): return total number of global nodes
        connectivity(): return element connectivity matrix, left to right
        x(): x-vector for evaluation of functions over entire mesh
    '''
    def __init__(self, C, a, b):
        '''
        PARAMETERS
            C: number of cells in mesh
            a, b: left and right ends of mesh
        '''
        self.__a = a # Left endpoint
        self.__b = b # Right endpoint
        self.__C = int(C) # Number of cells
        self.__N = int(C + 1) # Number of nodes
        self.__c = np.abs(b - a) # Domain length 
        self.__h = self.__c/C # Element length, assume all equal

    def all_nodes(self):
        '''Return ordered array of all nodes in mesh'''
        return np.linspace(0, self.__N - 1, num = self.__N) # List of nodes
    
    def cell_nodes(self, e):
        '''
        Return all nodes of cell e. Only linear elements for now.
        '''
        return (e, e+1)
    
    def all_coords(self):
        '''
        RETURNS: ordered array of mesh coordinates. Each tuple represents 
        coordinates of one cell. Only evenly spaced meshes for now.
        '''
        return np.array([[self.__a+e*self.__h, self.__a+e*(self.__h)+1]
                         for e in range(self.__C) ])
    
    def cell_coords(self, e):
        '''
        RETURNS: coordinates of nodes of e^th cell (only linear for now)
        '''
        return np.array([self.__a+e*self.__h, self.__a+(e+1)*(self.__h)])
        
    def node_coord(self, n):
        '''RETURNS: coordinate of n^th node'''
        return self.__a + n*self.__h
    
    def node_coords(self):
        '''RETURNS: coordinates of all global nodes'''
        return np.array([self.__a + n*self.__h for n in self.all_nodes()])
    
    def cells(self):
        '''RETURNS: ordered array of cell numbers'''
        return np.linspace(0, self.__C-1, num = self.__C)
        
    def size(self):
        '''RETURNS: number of cells in mesh'''
        return self.__C
    
    def nnodes(self):
        '''RETURNS: number of nodes'''
        return self.__N
    
    def connectivity(self):
        '''RETURNS: element connectivity matrix, left to right'''
        return np.hstack((np.atleast_2d(self.all_nodes()[:self.size()]).T, 
                          np.atleast_2d(self.all_nodes()[:self.size()]+1).T))
    
    def x(self, n=10):
        '''RETURNS: x-vector of length of mesh
        PARAMETERS:
            n: number of points in vector, defaults to 10
        '''
        return np.linspace(self.__a, self.__b, self.__C*n)
 


## Linear Element

In [ ]:
class Element1D:
    '''
    Defines a single 1D finite element, with either linear or 
    Quadratic Lagrange polynomial basis functions(only linear for now)
        
    PUBLIC METHODS
        ends(): Return global coordinates of element end points
        dof_points(): Return coordinates of locations for DOFs 
                      (same as end points of element for linear case)
        dof_nos(): Return array of DOF numbers
        num_dofs(): Return number of degrees of freedom 
        eval(n, x): Evaluate phi_n(x), i.e. evaluate the n_th 
                    element basis function 
                    in terms of the global x-coord (not xi)
        dx(n, x): Evaluate first derivative of basis function 
                  dphi[n](x) (not xi)
        integrate(n, f1, f2, include_deriv): return an array of values 
            for the integral $\int_e f_1(x)f_2(x)\phi_n(x)dx$,
            one value for each n. The f's are optional. if 
            include_deriv=True then $\phi^\prime$ is put in the integral.
    '''
    def __init__(self, mesh, bfuncs, e, dofs):
        '''
        PARAMETERS
            mesh: the mesh on which the element is located
            bfuncs: the BasisFunctions instance
            e: element number (in mesh.cells() array)
            ends: a tuple of ints giving the numbers of the 
                  endpoints of this element in the mesh
            dofs: array indexing the numbers of the dofs
        '''
        self.__e = e
        ends = mesh.cell_nodes(e)
        self.__endcoords = mesh.cell_coords(e)
        self.__numdofs = bfuncs.size()
        self.__dofs = dofs
        self.__dofcoords = np.linspace(self.__endcoords[0],
                                       self.__endcoords[1],self.__numdofs)
        self.__bfuncs = bfuncs
        self.__mesh = mesh
        self.__bfuncs = bfuncs
    
    # Getter methods for finite element endpoints, dof coordinates etc.
    def endcoords(self):
        '''Get end points of element'''
        return self.__endcoords
    
    def dofcoords(self):    
        '''
        Get coordinates of degree of freedom points of element. 
        These are the same as the node coordinates for linear element
        '''
        return self.__mesh.cell_coords(self.__e)
    
    def dofs(self):    
        '''Get indices of degrees of freedom. 
        Note these are same as node numbers for a linear element'''
        return self.__dofs
    
    def numdofs(self):
        '''Get number of degrees of freedom, same as 
        number of basis functions'''
        return self.__numdofs 
    
    def length(self):
          return self.endcoords()[1] - self.endcoords()[0]
    
    def e(self):
        '''RETURNS: element index number in mesh'''
        return self.__e
    
    # Evaluate shape functions over the element in terms of 
    # global coordinate x
    def phi(self, n, x):
        '''Evaluate $\phi_n$ on the element. First map global to 
        local coordinates, then use BasisFunctions1D public 
        eval() method to return resulting array'''
        xi = (x - self.__endcoords[0])/self.length()
        # Return the evaluation on xi, but only for 0 <= xi <= 1
        return self.__bfuncs.eval(n, xi) * (xi >= 0) * (xi <= 1)
    
    def phi_prime(self, n, x):
        '''Evaluate $d\phi_n/dx$ on the element. First map global 
        to local coordinates, then use BasisFunctions1D 
        public dx() method to return resulting array'''
        xi = (x - self.__endcoords[0])/self.length()
        # Return the derivative on xi, but only for 0 <= xi <= 1
        return self.__bfuncs.dx(n, xi) * (xi >= 0) * (xi <= 1)    
    
    def integrate(self, x, f1=None, f2=None, deriv=False, c=None):
        '''Define integration method over the element. 
        Used when defining the SolutionSpace1D.
        PARAMETERS
            x: global x-vector
            c: a constant to multiply the integral with
            f1: optional function, must be mapped to element coordinates 
                (i.e. must be a vector)
            f2: see f1
            deriv: boolean to decide whether $\phi$ or $\phi^\prime$ 
                   is used in the integral
        RETURNS
            I: array of integral values, one for each basis function
        '''
        L = self.length()
        xi = (x - self.__endcoords[0])/L
        
        if f1 is None:
            f1 = np.ones_like(xi)
        if f2 is None:
            f2 = np.ones_like(xi)
        if c is None:
            c = np.ones_like(xi)
        if not deriv:
            f = np.array([f1*f2*self.__bfuncs.eval(n,xi) 
                          for n in range(self.numdofs())]) 
        else:
            f = np.array([f1*f2*self.__bfuncs.dx(n,xi)/L 
                          for n in range(self.numdofs())])
        I = np.trapz(f,xi)
        return I

## Solution Space

In [ ]:
class SolutionSpace1D:
    '''Defines a complete trial solution function space 
    over a mesh of elements. Building blocks are Mesh1D 
    and BasisFunctions1D objects. 
    
    PUBLIC METHODS
        size(): Return number of elements
        ndofs(): Return sum of all dofs over all elements
        all_dofcoords(): Return array of coordinates 
                         of all dofs
        dof_coord(n): Return coordinate of dof n
        int_gal(f, deriv): Return stiffness matrix based on 
                           integration of Galerkin criterion
    '''
    def __init__(self, mesh, bfuncs):
        '''
        PARAMETERS
            mesh: a Mesh1D object
            bfuncs: a BasisFunctions1D object
        '''
        self.__mesh = mesh
        self.__bfuncs = bfuncs
        self.__size = mesh.size()
        # List to be populated with elements in a loop below
        self.__elems = list([])
        # List to be populated with DOF coordinates in the same loop
        self.__dofcoords = list([]) 
        self.__ndofs = 0 # Initialise counter to zero
        
        # Populate solution space with elements 
        #(cells and associated basis functions)
        for e in range(self.__size):
            # Boundary conditions are Dirichlet at 
            # beginning and end of mesh for now
            if e == 0:
                self.__ndofs += int(self.__bfuncs.size())
                # Note this would be [2*e, 2*e+1, 2*e+2] for 
                # quadratic element
                dofs = [e, e+1] 
                # This would be range(3) for quadratic elements
                newdofs = range(2) 
            else:
                self.__ndofs += int(self.__bfuncs.size() - 1)
                dofs = [e, e+1]
                # This would be range(1,3) for quadratic elements
                newdofs = range(1,2) 
            # Create new element in mesh
            elem = Element1D(self.__mesh, self.__bfuncs, e, dofs) 
            # Add new element to the array of elements
            self.__elems.append(elem) 
            for i in newdofs:
                self.__dofcoords.append(elem.dofcoords()[i])
                            
    # Getter methods
    def bfuncs(self):
        return self.__bfuncs
    def size(self):
        return self.__size
    def ndofs(self):
        return self.__ndofs
    def dofcoords(self):
        return self.__dofcoords
    def x(self):
        return self.__mesh.x()
    
    def int_gal(self, f=None, deriv=[False, False]):
        '''Integration method for Galerkin criterion. 
        Uses the Element1D `integrate` method.
        PARAMETERS
            deriv: Pair of booleans, giving the ability to 
                   mix and match $\phi$ and $\phi^\prime$
            f: Optional function that may be included in the derivative
        RETURNS
            A: Combined stiffness matrix for all elements
        '''
        # Stiffness matrix to be populated
        A = np.zeros([self.__ndofs, self.__ndofs]) 
        # Loop over all elements and add integrated terms into 
        # stiffness matrix
        for elem in self.__elems:
            # List of global degree of freedom indices on current element
            d = elem.dofs() 
            # Number of degrees of freedom on current element, this will 
            # be same as number of element equations
            D = elem.numdofs()  
            # Length of element, for conversion from local to 
            # global coordinates                    
            L = elem.length() 

            for i in range(D):
                if deriv[1]:
                    phi = elem.phi_prime(i, self.x())/L
                    # Because of the chain rule d(xi)/dx = 1/L
                else:
                    phi = elem.phi(i, self.x())
                A[d,d[i]] += elem.integrate(self.x(), 
                                            phi,f,deriv=deriv[0])
                                                                            
        return A

## Galerkin Solution

In [ ]:
class Galerkin:
    '''Galerkin solution for a linear, first order homogeneous ODE. 
    Currently only accepts Dirichlet
    boundary conditions at either end of interval.
    
    PUBLIC METHODS
    get_K(): returns the integrated left hand side of the 
             Galerkin criterion (i.e. the global stiffness matrix)
    apply_bc(): applies boundary conditions to stiffness matrix
    solve(): uses np.linalg.solve to solve the system for nodal unknowns
    plot(): plot solution and exact solution, if available
    '''
    def __init__(self, solspace, terms, derivs, 
                 bc=None, exact_soln=None, rhs=None):
        '''
        PARAMETERS
            solspace: solution space in which to evaluate the integral
            terms: number of terms in each integral
            derivs: 2D array of size terms x 2, each row is a pair of 
                    booleans, where True gives $\phi_prime$
                    and False gives $\phi$
            bc: array of pairs specifying Dirichlet BCs as [node, value]
            rhs: forcing term on rhs of Galerkin criterion
            exact_soln: function defining the exact solution 
                        over the domain
        '''
        self.__dofcoords = solspace.dofcoords()
        self.__exact_soln = exact_soln
        self.__bc = bc
        self.__rhs = rhs
        self.__N = solspace.ndofs()
        self.__x = solspace.x()
        self.__bfuncs = solspace.bfuncs()
        if exact_soln is not None: 
            self.__y = exact_soln(solspace.x())
        
        # Find full stiffness matrix 
        # (before application of boundary conditions)
        self.__K_full = 0*solspace.int_gal(deriv=[False, False])
        for i in range(terms):
            self.__K_full += solspace.int_gal(deriv=derivs[i])        
        
    def get_K(self):
        '''Get stiffness matrix
        RETURNS: Numerically integrated lhs of the Galerkin criterion'''
        return self.__K_full
    
    def apply_bc(self):
        '''Apply Dirichlet boundary conditions. 
        RETURNS: final stiffness matrix and right-hand vector'''
        # Make stiffness matrix
        K = self.get_K()
        K[0] = 0
        K[0,0] = 1
        K[-1] = 0
        K[-1, -1] = 1
        b = np.zeros(self.__N)
        b[0] = self.__bc[0][1]
        b[-1] = self.__bc[-1][1]
        return (K, b)
    
    def solve(self):
        '''Solve system equations. 
        RETURNS 
            Vector of approximate nodal values of field variable'''
        (K, b) = self.apply_bc()
        return np.round(np.linalg.solve(K, b), 3)       
    
    def plot(self):
        '''Plot approximation over domain, 
        along with exact solution where available'''
        plt.figure(figsize=(12,8))
        plt.plot(self.__dofcoords, self.solve())
        plt.plot(self.__x, self.__exact_soln(self.__x))

# Validation Code
The following code blocks test the classes defined above on the 1D steady heat equation with no forcing (RHS) terms. 

In [ ]:
# Validate solver on 1D steady heat equation
def heat1D(elements):
    '''Validation function for finite element code.
    Uses 1D steady heat equation with 
    Dirichlet boundary conditions.
    PARAMETERS
        elements: number of elements in mesh
    '''
    
    # Create BasisFunctions1D object, use linear basis functions
    lin_bfuncs = BasisFunctions1D(1)
    # Create Mesh1D object, domain [0, 2]
    mesh = Mesh1D(elements,0.0,2.0)
    # Create SolutionSpace1D object
    solspace = SolutionSpace1D(mesh,lin_bfuncs)
    # Define exact solution
    T = lambda x, k=1: (2*(np.exp(2/k) - 2) + 
                        2 * np.exp(x/k)) / (np.exp(2/k) - 1)
    # Create and plot Galerkin solution
    gal = Galerkin(solspace, 2, 
                   [[False, True], [True, True]],[[0,2],[2,4]], T)
    gal.plot()

def heat_interact(f):
    """ Streamline Interact function for T_approx """
    interactive_plot = interactive(f, elements=(1,8,1))
    output = interactive_plot.children[1]
    output.layout.height = '500px'
    return interactive_plot

def plot_bf(order):
    '''Plot basis functions
    PARAMETERS:
    order: 1 for linear, 2 for quadratic'''
    xi = np.linspace(0, 1, num=10)
    lin_bfuncs = BasisFunctions1D(order)
    lin_bfuncs.plot(xi)
    
def bf_interact(f):
    '''Streamline Interact function for plot_bf'''
    interactive_plot = interactive(f, order=(1,2,1))
    output = interactive_plot.children[1]
    output.layout.height = '500px'
    return interactive_plot  

In [ ]:
# Interactive plot of solution with 
# slider for number of elements
heat_interact(heat1D)

In [ ]:
# Interactive plot of basis functions
# with slider for order
bf_interact(plot_bf)